# different strategy for different variables using 

#simpleimputer
#Pipeline 
#column transformer


In [109]:
import numpy as np 
import pandas as pd
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [110]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

print('shape of train df',train.shape)
print('shape of test df',test.shape)


shape of train df (1460, 81)
shape of test df (1459, 80)


In [133]:
X_train=train.drop(columns="SalePrice",axis=1)
y_train=train['SalePrice']
X_test=test.copy()

print('shape of X_train',X_train.shape)
print('shape of y_train',y_train.shape)
print('shape of X_test',X_test.shape)


shape of X_train (1460, 80)
shape of y_train (1460,)
shape of X_test (1459, 80)


# missing value imputation 


In [112]:
isnull_sum=X_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [113]:
num_vars=X_train.select_dtypes(include=['int64','float64']).columns
num_vars_miss=[var for var in num_vars if isnull_sum[var]>0]

In [114]:
num_vars_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

# finding the categorical variable which have missing value 

In [115]:

cat_vars=X_train.select_dtypes(include=["O"]).columns
cat_vars_miss=[var for var in cat_vars if isnull_sum[var]>0]

In [116]:
cat_vars_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [117]:
num_var_mean=['LotFrontage']   #inserting mean value 

In [118]:
num_var_median=['MasVnrArea', 'GarageYrBlt'] #inserting median values 

In [119]:
cat_vars_mode=['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',]  #inserting mode value 

In [120]:
cat_vars_missing=[ 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']   #inserting constant value 

In [121]:
#Pipeline 

In [122]:
num_var_mean_imputer = Pipeline(steps=[('imputer',SimpleImputer(strategy='mean'))])
num_var_median_imputer = Pipeline(steps=[('imputer',SimpleImputer(strategy='median'))])
cat_vars_mode_imputer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))])
cat_vars_missing_imputer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value='missing'))])

In [139]:
preprocessor= ColumnTransformer( transformers=[
                                ('mean_imputer',num_var_mean_imputer,num_var_mean),
                                ('median_imputer',num_var_median_imputer,num_var_median),
                                ('mode_imputer',cat_vars_mode_imputer,cat_vars_mode),
                                ('missing_imputer',cat_vars_missing_imputer,cat_vars_missing)
                                              ] )

In [124]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_imputer',
                                 Pipe

In [128]:
preprocessor.named_transformers_["mean_imputer"].named_steps["imputer"].statistics_

array([70.04995837])

In [129]:
train['LotFrontage'].mean()

70.04995836802665

In [135]:
preprocessor.named_transformers_["mode_imputer"].named_steps["imputer"].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [131]:
X_train_clean=preprocessor.transform(X_train)
X_test_clean=preprocessor.transform(X_test)

In [132]:
X_train_clean

array([[65.0, 196.0, 2003.0, ..., 'missing', 'missing', 'missing'],
       [80.0, 0.0, 1976.0, ..., 'missing', 'missing', 'missing'],
       [68.0, 162.0, 2001.0, ..., 'missing', 'missing', 'missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'missing', 'missing', 'missing'],
       [75.0, 0.0, 1965.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [134]:
X_test_clean

array([[80.0, 0.0, 1961.0, ..., 'missing', 'MnPrv', 'missing'],
       [81.0, 108.0, 1958.0, ..., 'missing', 'missing', 'Gar2'],
       [74.0, 0.0, 1997.0, ..., 'missing', 'MnPrv', 'missing'],
       ...,
       [160.0, 0.0, 1960.0, ..., 'missing', 'missing', 'missing'],
       [62.0, 0.0, 1980.0, ..., 'missing', 'MnPrv', 'Shed'],
       [74.0, 94.0, 1993.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [138]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('missing_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   

In [141]:
X_train_clean_miss_var=pd.DataFrame(X_train_clean,columns=num_var_mean+num_var_median+cat_vars_mode+cat_vars_missing)

In [142]:
X_train_clean_miss_var

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,2003.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,missing,missing,missing
1,80.0,0.0,1976.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
2,68.0,162.0,2001.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
3,60.0,0.0,1998.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,missing,missing,missing
4,84.0,350.0,2000.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,0.0,1999.0,Grvl,None,Gd,TA,No,Unf,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
1456,85.0,119.0,1978.0,Grvl,Stone,Gd,TA,No,ALQ,Rec,SBrkr,TA,Attchd,Unf,TA,TA,missing,MnPrv,missing
1457,66.0,0.0,1941.0,Grvl,None,TA,Gd,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,missing,GdPrv,Shed
1458,68.0,0.0,1950.0,Grvl,None,TA,TA,Mn,GLQ,Rec,FuseA,Gd,Attchd,Unf,TA,TA,missing,missing,missing


In [143]:
X_train_clean_miss_var.isnull().sum().sum()

0

In [145]:
train['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [146]:
X_train_clean_miss_var['Alley'].value_counts()

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [147]:
X_train_clean_miss_var['MiscFeature'].value_counts()

missing    1406
Shed         49
Gar2          2
Othr          2
TenC          1
Name: MiscFeature, dtype: int64